In [123]:
import yfinance as yf
import pandas as pd
import sys

In [124]:
# Input parameters
symbol = 'VAS.AX'
period= '5y'
invest_interval = 28 # Days
recur_invest = 1000

In [125]:
# Get symbol data
hist = yf.Ticker(symbol).history(period) # Retrieves pd.DataFrame
hist[0:5] # To display some values

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
Date,,,,,,,,
2018-07-16 00:00:00+10:00,79.650002,79.669998,79.239998,79.279999,73177,0.0,0.0,0.0
2018-07-17 00:00:00+10:00,79.089996,79.279999,78.769997,78.809998,116396,0.0,0.0,0.0
2018-07-18 00:00:00+10:00,79.239998,79.489998,79.190002,79.349998,33198,0.0,0.0,0.0
2018-07-19 00:00:00+10:00,79.440002,79.739998,79.300003,79.519997,94346,0.0,0.0,0.0
2018-07-20 00:00:00+10:00,79.529999,80.000000,79.489998,79.839996,79711,0.0,0.0,0.0


In [126]:
# Get every nth average value between high and low
hist['avg'] = hist[['High', 'Low']].mean(axis=1).round(2)
invest_days = hist.iloc[0::invest_interval, :]
invest_days = invest_days['avg'].reset_index() # This makes the date columns accessible and now we are just focusing on date and avg
invest_days[0:5] # To display some values

,Date,avg
0,2018-07-16 00:00:00+10:00,79.45
1,2018-08-23 00:00:00+10:00,80.02
2,2018-10-02 00:00:00+10:00,78.10
3,2018-11-09 00:00:00+11:00,75.22
4,2018-12-19 00:00:00+11:00,71.04


In [131]:
# Get parameters for calculations
start_date = invest_days.iloc[0, 0].date()
end_date = invest_days.iloc[-1, 0].date()
last_price = invest_days.iloc[-1, 1]

def format_dollar(dol: int) -> str:
    if dol < 0:
        dol = abs(dol)
        return '-${:,.2f}'.format(dol)
    else:
        return '${:,.2f}'.format(dol)

amount_not_invested = amount_invested = total_end_value = total_end_gain = 0
for i in range(len(invest_days)):
    avg = invest_days.iloc[i, 1]
    stocks_bought = int(recur_invest // avg)
    cur_value = round(stocks_bought*avg, 2)
    
    # Some amount of money will simply not be invested because you cannot buy a portion of a stock (in this model)
    amount_not_invested += recur_invest - cur_value
    amount_invested += cur_value
    
    end_value = stocks_bought*last_price
    total_end_value += end_value
    total_end_gain += end_value - cur_value
    
    avg = format_dollar(avg)
    cur_value = format_dollar(cur_value)
    print('On {date} bought {number} of {symbol} at {price}, investing {value}'.format(
        date=invest_days.iloc[i, 0].date(),
        number=stocks_bought,
        symbol=symbol,
        price=avg,
        value=cur_value))

amount = format_dollar(recur_invest)
last_price = format_dollar(last_price)
amount_invested = format_dollar(amount_invested)
amount_not_invested = format_dollar(amount_not_invested)
total_end_value = format_dollar(total_end_value)
total_end_gain = format_dollar(total_end_gain)
print('''
Began buying stock on {start_date} with an interval of {interval} days per investment of {amount}.
Sold all stock on {end_date} for a price of {last_price}.
Amount of money that was not invested (cannot buy portion of a stock in this model): {not_invested}
Amount of money that was invested: {invested}
The total value of all investment: {total_value}
The total gain from investment: {total_gain}
'''.format(
    start_date=start_date,
    interval=invest_interval,
    amount=amount,
    end_date=end_date,
    last_price=last_price,
    not_invested=amount_not_invested,
    invested=amount_invested,
    total_value=total_end_value,
    total_gain=total_end_gain))

On 2018-07-16 bought 12 of VAS.AX at $79.45, investing $953.40
On 2018-08-23 bought 12 of VAS.AX at $80.02, investing $960.24
On 2018-10-02 bought 12 of VAS.AX at $78.10, investing $937.20
On 2018-11-09 bought 13 of VAS.AX at $75.22, investing $977.86
On 2018-12-19 bought 14 of VAS.AX at $71.04, investing $994.56
On 2019-02-01 bought 13 of VAS.AX at $74.24, investing $965.12
On 2019-03-13 bought 12 of VAS.AX at $78.85, investing $946.20
On 2019-04-24 bought 12 of VAS.AX at $80.76, investing $969.12
On 2019-06-04 bought 12 of VAS.AX at $80.70, investing $968.40
On 2019-07-15 bought 11 of VAS.AX at $84.22, investing $926.42
On 2019-08-22 bought 12 of VAS.AX at $82.87, investing $994.44
On 2019-10-01 bought 11 of VAS.AX at $85.12, investing $936.32
On 2019-11-08 bought 11 of VAS.AX at $85.23, investing $937.53
On 2019-12-18 bought 11 of VAS.AX at $87.24, investing $959.64
On 2020-01-31 bought 11 of VAS.AX at $88.93, investing $978.23
On 2020-03-11 bought 13 of VAS.AX at $74.87, investing 